# Rescession and housing prices in University towns 

**Hypothesis**: University towns (cities which have a high percentage of university students compared to the total population of the city) have their mean housing prices less effected by recessions. A Recession is defined as starting with two consecutive quarters of Gross Domestic Product (GDP) decline, and ending with two consecutive quarters of GDP growth.

**Methodology**:

For the years 2000 - 2016 I want to compute the ratio of the mean price of houses the quarter before the recession starts compared to the recession bottom (as a measure for market loss). I want to compare these ratios for univercity towns versus no univercity towns using a t-test. 

The following data files are used:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. 

**Results**: The null-hypothesis (no difference between the groups) is rejected with a p-value of 0.00. There is a lower mean price ratio (reduced market loss) in university towns.

In [2]:
#import libraries
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from scipy import stats

In [3]:
#creating a table of the univercity towns
df= pd.DataFrame(columns = ["State", "RegionName"])
unitowns = open("university_towns.txt")
state=[]
for line in unitowns:
    if "[edit]" in line:
        x = line.find("[")
        st = line[:x]
        st=st.rstrip()
        if st not in state:  
            df = df.append({"State": st}, ignore_index=True)
            state.append(st)
    else:
        y = line.find("(")
        rn= line[:y]
        rn=rn.rstrip()
        df = df.append({"RegionName": rn, "State": st}, ignore_index=True)
#drop missing values
df.dropna(axis=0, inplace = True) 
df.reset_index(drop=True, inplace=True)
df.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [4]:
#loading the datset for GPD Data
gpd= pd.read_excel("gdplev-1.xlsx", header = [5], index_col = [4])
gpd.dropna(axis = 0, how ="all", inplace = True)
gpd = gpd.loc["2000q1":, ["GDP in billions of chained 2009 dollars.1"]]
gpd.head()

,GDP in billions of chained 2009 dollars.1
2000q1,12359.1
2000q2,12592.5
2000q3,12607.7
2000q4,12679.3
2001q1,12643.3


In [5]:
# claculation of start, bottom and end of the recession
gpd["shift"] = gpd["GDP in billions of chained 2009 dollars.1"].shift(1)
gpd["shift2"] = gpd["GDP in billions of chained 2009 dollars.1"].shift(2)
gpd["shift3"] = gpd["GDP in billions of chained 2009 dollars.1"].shift(3)
gpd["three_smaller"] = (gpd["GDP in billions of chained 2009 dollars.1"] < gpd["shift"]) & (gpd["shift"] < gpd["shift2"]) & (gpd["shift2"] < gpd["shift3"])
gpd["twice_bigger"] = (gpd["GDP in billions of chained 2009 dollars.1"] > gpd["shift"]) & (gpd["shift"] > gpd["shift2"])
starts =[]
for i,row in gpd.iterrows():
    if row["three_smaller"] == True:
        st = gpd.iloc[gpd.index.get_loc(i)-2].name
        starts.append(st)
start= str(starts[0])
end = gpd.loc[start:]["twice_bigger"].idxmax() 
bottom = gpd.loc[start:end]["GDP in billions of chained 2009 dollars.1"].argmin()
print("start od recession:",start,"\n","end of recession:", end, "\n", "bottom of recession:", bottom)

start od recession: 2008q3 
 end of recession: 2009q4 
 bottom of recession: 2009q2


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  


In [6]:
#loading the housing prices dataset 
housing = pd.read_csv("City_Zhvi_AllHomes.csv", index_col = ["State", "RegionName"])
housing = housing.loc[:, "2000-01":]
housing.head()

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
CA,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
IL,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
PA,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
AZ,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [7]:
#convertig the data to quarters
for i in housing.columns:
    if "-" in i:
        x = i.split("-")
        y = x[1]
        z = x[0]
        zahl = int(y)
        if zahl <= 3:
            housing.rename(columns={i:"{}q1".format(x[0])}, inplace=True)
        elif zahl <=6:
            housing.rename(columns={i:"{}q2".format(x[0])}, inplace=True)
        elif zahl <=9:
            housing.rename(columns={i:"{}q3".format(x[0])}, inplace=True)
        else:
            housing.rename(columns={i:"{}q4".format(x[0])}, inplace=True)
housing_q = housing.groupby(by=housing.columns, axis=1).apply(lambda g: g.mean(axis=1))
housing_q.rename(index={'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'},inplace = True)
housing_q.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [8]:
#creating a list of the univercity towns

#some of the Univercity towns have the Univercity name before the city name. I want to split those from the citie-names, so i can compare the city-names with the other dataset
#e.g "Univercity District, San Bernardino"
liste = df["RegionName"].values
new_list = []
for i in liste:
    if "," in i:
        i = i.split(",")
        for z in i:
            z = z.strip()
            new_list.append(z)
    else:
        new_list.append(i)

In [9]:
#running the t-test comparing the price ratio of the beginning and the bottom of the regression
#for cat1 = no univercity town versus cat2 = univercity towns
housing_q.reset_index(inplace=True)
housing_q['unitown'] = housing_q['RegionName'].apply(lambda x: 1 if (x in new_list)  else 0)
housing_q["price_ratio"] = housing_q["2008q2"]/housing_q["2009q2"]
cat1 = housing_q[housing_q['unitown']==0]
cat1.dropna(inplace = True)
cat2 = housing_q[housing_q['unitown']==1]
cat2.dropna(inplace = True)
statistic,pvalue = stats.ttest_ind(cat1['price_ratio'], cat2['price_ratio'])
better = "university town" 
print(" p-value:", pvalue, "\n", "null-hypothesis rejected", "\n", "lower mean price ratio(reduced market loss) = university towns") 

 p-value: 0.0001157169756509147 
 null-hypothesis rejected 
 lower mean price ratio(reduced market loss) = university towns


C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
